In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

ROOT_DATA_DIR = os.path.join('..', 'Data', 'PCB_DATASET')
TEMPLATE_DIR = os.path.join(ROOT_DATA_DIR, 'PCB_USED') 
ALL_IMAGES_DIR = os.path.join(ROOT_DATA_DIR, 'images') 

OUTPUT_MASKS_DIR = os.path.join('..', 'Data', 'PROCESSED_DEFECT_MASKS') 
os.makedirs(OUTPUT_MASKS_DIR, exist_ok=True)

TEMPLATE_INDICES = ['01', '04', '05', '06', '07', '08', '09', '10', '11', '12']
DEFECT_TYPES = ['Missing_hole', 'Mouse_bite', 'Open_circuit', 'Short', 'Spur', 'Spurious_copper']

In [ ]:

def process_image_pair(template_idx, defect_type, test_img_name):
    """Loads, aligns, subtracts, and thresholds a single PCB pair."""

    
    template_path = os.path.join(TEMPLATE_DIR, f'{template_idx}.JPG')
    test_img_path = os.path.join(ALL_IMAGES_DIR, defect_type, test_img_name)


    template = cv2.imread(template_path, 0)
    test_img = cv2.imread(test_img_path, 0)

    if template is None or test_img is None:
        print(f"Skipping {test_img_name}: Image not found.")
        return None

 
    orb = cv2.ORB_create(nfeatures=5000)
    kp1, des1 = orb.detectAndCompute(template, None)
    kp2, des2 = orb.detectAndCompute(test_img, None)

    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    good_matches = sorted(matches, key = lambda x:x.distance)[:50] 

    points1 = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    points2 = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    H, mask = cv2.findHomography(points2, points1, cv2.RANSAC, 5.0)
    height, width = template.shape
    aligned_test_img = cv2.warpPerspective(test_img, H, (width, height))

 
    difference_map = cv2.absdiff(template, aligned_test_img)
    ret, defect_mask_raw = cv2.threshold(difference_map, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    

    kernel = np.ones((3, 3), np.uint8)
    defect_mask = cv2.erode(defect_mask_raw, kernel, iterations=1)
    defect_mask = cv2.dilate(defect_mask, kernel, iterations=1)

    return template, test_img, aligned_test_img, defect_mask


for defect_type in DEFECT_TYPES:
    print(f"\nProcessing and Saving defect type: {defect_type}")
    current_defect_dir = os.path.join(ALL_IMAGES_DIR, defect_type)
    

    save_dir = os.path.join(OUTPUT_MASKS_DIR, defect_type)
    os.makedirs(save_dir, exist_ok=True) 


    test_image_files = [f for f in os.listdir(current_defect_dir) if f.endswith('.JPG') or f.endswith('.jpg')]

    for test_img_name in test_image_files:
        template_idx = test_img_name.split('_')[0] 
        
        if template_idx in TEMPLATE_INDICES:
            
 
            results = process_image_pair(template_idx, defect_type, test_img_name)
            
            if results is not None:
                template, test_img, aligned_test_img, defect_mask = results
                
 
                mask_filename = f"MASK_{test_img_name.replace('.JPG', '.png').replace('.jpg', '.png')}"
                save_path = os.path.join(save_dir, mask_filename)
                

                cv2.imwrite(save_path, defect_mask)
                
                if test_img_name.endswith('_02.JPG'):
                     plt.figure(figsize=(15, 4))
                     plt.suptitle(f"Saved MASK for Defect: {defect_type}, PCB: {template_idx}, Test: {test_img_name}", fontsize=12)

                     plt.subplot(1, 3, 1)
                     plt.imshow(test_img, cmap='gray')
                     plt.title("Original Test")

                     plt.subplot(1, 3, 2)
                     plt.imshow(aligned_test_img, cmap='gray')
                     plt.title("Aligned Test")

                     plt.subplot(1, 3, 3)
                     plt.imshow(defect_mask, cmap='gray')
                     plt.title(f"Final Defect Mask (Saved to {defect_type})")
                     plt.show()

print("\n--- All defect masks have been generated and saved! ---")


Processing and Saving defect type: Missing_hole

Processing and Saving defect type: Mouse_bite

Processing and Saving defect type: Open_circuit

Processing and Saving defect type: Short

Processing and Saving defect type: Spur

Processing and Saving defect type: Spurious_copper

--- All defect masks have been generated and saved! ---
